In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import penaltyblog as pb
from scipy.optimize import minimize
from scipy.stats import poisson

In [3]:
df = pb.footballdata.fetch_data("England", 2018, 0)
df[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]].head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,2018-08-10,Man United,Leicester,2,1
1,2018-08-11,Bournemouth,Cardiff,2,0
2,2018-08-11,Fulham,Crystal Palace,0,2
3,2018-08-11,Huddersfield,Chelsea,0,3
4,2018-08-11,Newcastle,Tottenham,1,2


### Poisson Model

In [4]:
pois = pb.poisson.PoissonGoalsModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

In [5]:
pois.fit()

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
pois

Module: Penaltyblog

Model: Poisson

Number of parameters: 41
Log Likelihood: -1065.077
AIC: 2212.154

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.362                -0.856              
Bournemouth          1.115                -0.555              
Brighton             0.634                -0.731              
Burnley              0.894                -0.595              
Cardiff              0.614                -0.592              
Chelsea              1.202                -1.135              
Crystal Palace       1.004                -0.839              
Everton              1.055                -0.978              
Fulham               0.626                -0.431              
Huddersfield         0.184                -0.507              
Leicester            1.0                  -0.939              
Liverpool            1.532                -1.683              
Man City         

In [7]:
pois.get_params()

{'attack_Arsenal': 1.3622182692354565,
 'attack_Bournemouth': 1.1151950638392283,
 'attack_Brighton': 0.6339238162809544,
 'attack_Burnley': 0.8936894280490494,
 'attack_Cardiff': 0.6136413243137976,
 'attack_Chelsea': 1.2024806358377873,
 'attack_Crystal Palace': 1.0043948012112704,
 'attack_Everton': 1.0548886842016354,
 'attack_Fulham': 0.6255475814984898,
 'attack_Huddersfield': 0.18430169225986576,
 'attack_Leicester': 0.9995578292065143,
 'attack_Liverpool': 1.5319802424266495,
 'attack_Man City': 1.5983141483364756,
 'attack_Man United': 1.248697315010879,
 'attack_Newcastle': 0.8048579264432804,
 'attack_Southampton': 0.8907128648860027,
 'attack_Tottenham': 1.2641636112230386,
 'attack_Watford': 1.0298583264833256,
 'attack_West Ham': 1.0258195592667356,
 'attack_Wolves': 0.9157568799895628,
 'defence_Arsenal': -0.8560871226786197,
 'defence_Bournemouth': -0.5553454067066441,
 'defence_Brighton': -0.7306942286847046,
 'defence_Burnley': -0.595423759614251,
 'defence_Cardiff': 

In [8]:
goals = pois.predict("Watford", "Wolves")
goals

Module: Penaltyblog

Class: FootballProbabilityGrid

Home Goal Expectation: 1.309141560879353
Away Goal Expectation: 1.2031157023116301

Home Win: 0.3907965326317279
Draw: 0.2688104777800772
Away Win: 0.3403929895714368

In [9]:
goals.home_draw_away

[0.3907965326317279, 0.2688104777800772, 0.3403929895714368]

### Dixon and Coles

In [10]:
dc = pb.poisson.DixonColesGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

In [11]:
dc.fit()

/opt/conda/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
dc

Module: Penaltyblog

Model: Dixon and Coles

Number of parameters: 42
Log Likelihood: -1064.943
AIC: 2213.886

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.36                 -0.858              
Bournemouth          1.115                -0.555              
Brighton             0.632                -0.733              
Burnley              0.897                -0.592              
Cardiff              0.615                -0.591              
Chelsea              1.205                -1.13               
Crystal Palace       1.007                -0.837              
Everton              1.054                -0.977              
Fulham               0.625                -0.433              
Huddersfield         0.18                 -0.507              
Leicester            0.996                -0.94               
Liverpool            1.534                -1.679              
Man City 

In [13]:
dc.get_params()

{'attack_Arsenal': 1.3601911791708892,
 'attack_Bournemouth': 1.1145307240461475,
 'attack_Brighton': 0.6321895363265042,
 'attack_Burnley': 0.8970977474233514,
 'attack_Cardiff': 0.614718346758065,
 'attack_Chelsea': 1.2054631766079373,
 'attack_Crystal Palace': 1.0066565931220084,
 'attack_Everton': 1.0541841709895725,
 'attack_Fulham': 0.6246566669840924,
 'attack_Huddersfield': 0.18023347766337813,
 'attack_Leicester': 0.9964723801388785,
 'attack_Liverpool': 1.5337965942611675,
 'attack_Man City': 1.5986872195471342,
 'attack_Man United': 1.2510996911128331,
 'attack_Newcastle': 0.8063670442934346,
 'attack_Southampton': 0.8966160731893246,
 'attack_Tottenham': 1.2590431240092954,
 'attack_Watford': 1.0305630591196495,
 'attack_West Ham': 1.0234853816377847,
 'attack_Wolves': 0.9139478135985553,
 'defence_Arsenal': -0.8578523243246394,
 'defence_Bournemouth': -0.5551112760231923,
 'defence_Brighton': -0.7332968897143658,
 'defence_Burnley': -0.5924217806361621,
 'defence_Cardiff':

In [14]:
goals = dc.predict("Watford", "Wolves")
goals

Module: Penaltyblog

Class: FootballProbabilityGrid

Home Goal Expectation: 1.3057700841978466
Away Goal Expectation: 1.2027271998317837

Home Win: 0.38473277025139113
Draw: 0.27955307378424477
Away Win: 0.33571415594807

In [15]:
goals.home_draw_away

[0.38473277025139113, 0.27955307378424477, 0.33571415594807]

### Rue Salvesen Model

In [16]:
rs = pb.poisson.RueSalvesenGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

In [17]:
rs.fit()

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
rs

Module: Penaltyblog

Model: Rue Salvesen

Number of parameters: 43
Log Likelihood: -1061.167
AIC: 2208.334

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.496                -0.988              
Bournemouth          1.27                 -0.705              
Brighton             0.559                -0.655              
Burnley              0.964                -0.654              
Cardiff              0.585                -0.556              
Chelsea              1.183                -1.129              
Crystal Palace       1.027                -0.852              
Everton              1.034                -0.974              
Fulham               0.653                -0.455              
Huddersfield         0.023                -0.359              
Leicester            0.978                -0.916              
Liverpool            1.445                -1.585              
Man City    

In [19]:
rs.get_params()

{'attack_Arsenal': 1.4960574633781003,
 'attack_Bournemouth': 1.2701540413261327,
 'attack_Brighton': 0.559186251363228,
 'attack_Burnley': 0.9644520899122194,
 'attack_Cardiff': 0.5847648397569006,
 'attack_Chelsea': 1.1828466188120765,
 'attack_Crystal Palace': 1.0273361069287597,
 'attack_Everton': 1.0335248035400801,
 'attack_Fulham': 0.6531864264818924,
 'attack_Huddersfield': 0.023109559960240708,
 'attack_Leicester': 0.977933119588144,
 'attack_Liverpool': 1.4451581320799645,
 'attack_Man City': 1.5025454369883477,
 'attack_Man United': 1.366845541477835,
 'attack_Newcastle': 0.720009733703693,
 'attack_Southampton': 0.9416226570416543,
 'attack_Tottenham': 1.2427047093744437,
 'attack_Watford': 1.096790079793436,
 'attack_West Ham': 1.06163359275858,
 'attack_Wolves': 0.8501387957342722,
 'defence_Arsenal': -0.9879049844176601,
 'defence_Bournemouth': -0.704968272653022,
 'defence_Brighton': -0.6545658993274335,
 'defence_Burnley': -0.6541299575160815,
 'defence_Cardiff': -0.55

In [40]:
probs = rs.predict("Watford", "Wolves")
probs

Module: Penaltyblog

Class: FootballProbabilityGrid

Home Goal Expectation: 1.3094663275662697
Away Goal Expectation: 1.2096084394688094

Home Win: 0.3843462702759874
Draw: 0.2787256663458056
Away Win: 0.3369280633610962

In [31]:
goals.home_draw_away

[0.3843462702759874, 0.2787256663458056, 0.3369280633610962]

### Market Probabilities

In [41]:
probs = rs.predict("Watford", "Wolves")

In [42]:
probs.total_goals("over", 2.5)

0.4610704441088047

In [46]:
probs.total_goals("under", 2.5)

0.5389295558740843

In [48]:
probs.asian_handicap("away", -1.5)

0.8246856221675609

In [47]:
probs.asian_handicap("home", 1.5)

0.17531437781532913

In [37]:
probs.home_draw_away

[0.3843462702759874, 0.2787256663458056, 0.3369280633610962]